In [ ]:
import sys, os, re, subprocess, time, glob

In [ ]:
import pandas as pd

## Seinfeld

In [ ]:
#os.system('wget http://www.seinfeldscripts.com/seinfeld-scripts.html')
os.listdir('./')

In [ ]:
episode_link_string = str(subprocess.check_output(['grep','-A 2','href\|Season','post_nlp_comp/seinfeld-scripts.html']))

In [ ]:
episode_link_string[3000:4000]

In [ ]:
episode_links = re.findall('href=\" ?((?:The|Male)\w*\\.html?)\">([^<]*)', episode_link_string)

In [ ]:
current_season, current_episode = 0, 1
eps_pos = 0
episode_links_w_season = []
for row in episode_link_string.split('align="middle"'):
    if re.search('Season (\d)', row) is not None:
        current_season = int(re.search('Season (\d)', row).groups()[0])
        #print(current_season)
        current_episode = 1
    if episode_links[eps_pos][0] in row:
        #print('========',row)
        #print(episode_links[eps_pos],current_season,current_episode)
        episode_links_w_season.append((episode_links[eps_pos][0],
                                       current_season, current_episode))
        current_episode += 1
        eps_pos += 1
    if eps_pos == len(episode_links):
        break

In [ ]:
[e for e in episode_links_w_season if e[2] == 1]

In [ ]:
os.system('wget http://www.seinfeldscripts.com/TheAbstinence.htm')

In [ ]:
#Can't handle two-line stage directions - second part
#  gets into speaker lines.

def process_seinfeld_episode(ep_raw):
    scene_count = 0
    ep_lines = []
    current_line = ''
    found_ad_break = False
    have_prescene_line = False
    for l in ep_raw:
        l = re.sub('\s*<p( align="left")?>', '', l).strip()
        l = re.sub('</p>', '', l)
        l = re.sub('<br>', '', l)
        l = re.sub('&quot;', '"', l)
        l = re.sub('&#146;', '\'', l)
        l = re.sub('&#133;', '...', l)
        l = re.sub('&#150;', '-', l)
        if scene_count > 0 and \
            ('BeginAd' in l or l == 'END OF ACT 1'):
            found_ad_break = True
        l = re.sub('<!.*--><.*-->', '', l)
        
        #print(l)
        if scene_count == 0 and \
            ('setting' in l.lower() or '% Opening' in l or \
             'First scene' in l or 'Opening scene' in l or \
             '[Scene' in l or \
             'Jerry\'s opening' in l or 'Jerry\'s standup' in l or \
             'pening monologue' in l.lower() or \
             l == 'Monologue:' or l == '[Montage]' or \
             'jerry\'s apartment' in l.lower() or '[George\'s Apartment]' in l or \
             l[:6].lower() == 'monk\'s' or \
             'ACT 1' in l or 'ACT ONE' in l or \
             l[:18].lower() == 'at the comedy club' or \
             #TheDeal needs this specific one
             'LEVITAN: Remind me to tell you' in l or \
             #TheGoodSamaritan
             '(Jerry is driving alone ' in l or \
             #TheLimo
             'Do you think that the people at the airport' in l or \
             #TheVirgin
             '% Jerry and George talking near a bar' in l or \
             #TheBizarroJerry
             '[JERRY and GEORGE are eating' in l or \
             #TheCalzone
             'George in a meeting at Yankee' in l or \
             #TheCartoon
             'Some street , Jerry and Kramer walking' in l or \
             #TheChickenRoaster
             '% George and Jerry are on the side' in l or \
             #ThePackage
             'Elaine at the doctor\'s' in l or \
             #TheRye
             'ELAINE: [thinking] I can\'t believe' in l or 
             #TheSerenityNow
             '% The Costanzas are driving in the car' in l or \
             #TheSniffingAccountant
             'Jerry, George and Elaine at Monk\'s Caf' in l or \
             #TheVoice
             '(Scene: In the' in l or \
             #ThePostponement
             '[Jerry: and Elaine: are walking down the street' in l or \
             #TheSlicer
             '[Elaine wakes up in bed with Jerry.]' in l or \
             l.strip()[:4] == 'INT.'):
            scene_count += 1
        elif scene_count > 0 and \
            ('END OF SHOW' in l or '[End]' in l or \
            l[:7] == 'THE END' or l == 'END' or l == 'END!' or \
            'Spellchecked and reformatted' in l or \
            'Spell checked and reformatted' in l or \
            'InstanceEndEditable' in l):
            break

        if scene_count >= 1:
            if re.match('[\s\w\.\(\)]*:', l) is not None:
                if current_line != '':
                    #print(current_line)
                    ep_lines.append(current_line)
                current_line = l.strip()
            elif len(l) > 0 and l[0] not in ['[','(','%','=','*'] and \
                l[:4] not in ['INT.','EXT.','ACT '] and \
                l[:10] != 'END OF ACT' and \
                current_line != '':
                current_line = ' '.join([current_line, l.strip()])
            elif l[:4] in ['INT.','EXT.'] and ('ESTABLISHING' in l or 'ESTABLSIHING' in l):
                #this isn't marked as new scene; we want lines after this but before the new
                #   scene start to be put after the new scene start
                #but write anything in current_line first
                if current_line != '':
                    ep_lines.append(current_line)
                    current_line = ''
                have_prescene_line = True
            else: 
                #print(l)
                if '[setting' in l.lower() or '[New scene' in l.lower() or \
                    '[scene' in l.lower() or \
                    (l[:4] in ['INT.','EXT.'] and 'ESTABLISHING' not in l and 'ESTABLSIHING' not in l):
                    if current_line != '' and not have_prescene_line:
                        ep_lines.append(current_line)
                        current_line = ''
                    ep_lines.append('Scene %i' % scene_count)
                    if current_line != '' and have_prescene_line:
                        ep_lines.append(current_line)
                        current_line = ''
                        have_prescene_line = False
                    scene_count += 1
                    found_ad_break = False  #don't make another new scene below if it was about to do so

            #wait until we've written the current speaker line 
            #  before inserting the scene break
            if found_ad_break and re.match('[\s\w\.]*:', l) is not None:
                ep_lines.append('Scene %i' % scene_count)
                scene_count += 1
                found_ad_break = False
            
            #It still makes the odd mistake around the ad break I think

    #catch last line
    if current_line != '':
        ep_lines.append(current_line)
    
    return ep_lines

In [ ]:
with open('TheAbstinence.htm', 'r') as file:
    ep_raw = file.readlines()

In [ ]:
ep_lines = process_seinfeld_episode(ep_raw)
print('Found {:d} lines and {:d} scenes'.format(len(ep_lines), 
                                len([l for l in ep_lines if l[:5]=='Scene'])))

In [ ]:
ep_lines

In [ ]:
#Ran this once without writing refmt then went back and fixed function
#  to catch the problem ones
for ep in episode_links_w_season:
    
    #Formatting is bad so skip 7, and one is incomplete
    if ep[0] in ['TheBris.htm','TheParkingGarage.htm',
                 'TheStall.htm','TheSuzie.htm',
                 'TheNonFatYogurt.html','TheBizarroJerry.htm',
                 'TheChineseWoman.htm',
                 'TheWaitOut.htm']:
        continue
    
    os.system('wget http://www.seinfeldscripts.com/{:s}'.format(ep[0]))
    
    with open(ep[0], 'r') as file:
        ep_raw = file.readlines()
        
    ep_lines = process_seinfeld_episode(ep_raw)
    print('{:s}: Found {:d} lines and {:d} scenes'.format(ep[0],len(ep_lines),
                                                          len([l for l in ep_lines if l[:5]=='Scene'])))
    
    if len(ep_lines) > 150:
        with open('seinfeld_transcripts/refmt_{:02d}{:02d}.txt'.format(
            ep[1], ep[2]), 'w') as f:
            for line in ep_lines:
                f.write(line+'\n')
        
        os.system('rm {:s}'.format(ep[0]))

## Married With Children

In [ ]:
os.system('wget http://albundy.net/marriedaniac/ate/script.html')

In [ ]:
episode_link_string = str(subprocess.check_output(['grep','href="script','script.html']))

In [ ]:
episode_links = re.findall('href=\"(script/\w*\\.html)\">(\d{4})', episode_link_string)
episode_links = [e for e in episode_links if 'fst' not in e[0] and \
                 'tbl' not in e[0] and 'snd' not in e[0]]
episode_links = [e for e in episode_links if e[0] not in \
                 ['script/0101snd.html',
                  'script/0101fin.html',
                  'script/0101o.html',
                  'script/0206fin.html',
                  'script/0210fin.html',
                  'script/0307fin.html',
                  'script/0406fin.html',
                  'script/0407fin.html',
                  'script/0410fin.html',
                  'script/0507fin.html']]
#skip some with extra html
episode_links = [e for e in episode_links if e[0] not in ['script/0205t.html']]

In [ ]:
with open('0401t.html', mode='r', encoding='utf-8', errors='ignore') as f:
    ep_raw = f.readlines()

In [ ]:
#I may not have run this unedited for all episodes at once - took two batches I think
#  with a small change to finding speakers

def process_marriedwc_episode(ep_raw):
    scene_count = 0
    ep_lines = []
    current_line = ''
    found_ad_break = False
    for l in ep_raw:
        l = re.sub('<u>', '', l)
        l = re.sub('</u>', '', l)
        l = re.sub('^\n', '', l)
        if len(l) == 0:
            continue
        
        #print(l)
        if scene_count == 0 and 'ACT ONE' in l:
            scene_count += 1
        elif scene_count > 0 and \
            ('END OF SHOW' in l or '[End]' in l or \
            l[:7] == 'THE END' or l == 'END' or l == 'END!'):
            break

        if scene_count >= 1:
            #print(l)
            #Find the start of a spoken line
            if ((re.match('(^[A-Z\s0-9\.]{2,20})\)?\s+', l) is not None) or \
                re.match('[\s\w\.\(\)]*:', l) is not None) and \
                l[:3] != '   ' and l[0] != '\t' and \
                l[:6] != 'SCENE ' and l[:4] != 'ACT ' and \
                l[:5] not in ['FLIP ','FADE '] and l[:9] != 'DISSOLVE ' and \
                l[:8] != 'FRIENDS:' and l[:6] != 'WORKER':
                if current_line != '':
                    #print(current_line)
                    ep_lines.append(current_line)
                
                if re.match('^[\s\w\.\(\)]*:', l) is not None:
                    l = re.sub('\s+',' ', l)
                #if re.match('^[A-Z\.]{2,20}', l) is not None and '  ' in l:
                else:
                    speaker = re.match('(^[A-Z\s0-9\.]{2,20})\)?\s+', l).groups()[0].strip()
                    #can get '(AL   I) am' as speaker
                    if '  ' in speaker or '\t' in speaker:
                        speaker = re.sub('\t','  ', speaker)
                        speaker = re.sub('\s{2,20}','  ', speaker)
                        speaker, extra_bit = speaker.split('  ')[:2]
                        extra_bit = extra_bit.strip()
                        l = re.sub('(^[A-Z\s0-9\.]{2,20})\)?\s+',speaker+': '+extra_bit+' ', l)
                    else:
                        l = re.sub('(^[A-Z\s0-9\.]{2,20})\)?\s+',speaker+': ', l)
                    #print(speaker)
                
                current_line = l.strip()
            elif (l[:3] == '   ' or l[0] == '\t') and current_line != '':
                current_line = ' '.join([current_line, l.strip()])
            else:
                #print(l, l[0]=='\n', l[0]=='\t')
                if 'SCENE' in l:
                    if current_line != '':
                        ep_lines.append(current_line)
                        current_line = ''
                    ep_lines.append('Scene %i' % scene_count)
                    scene_count += 1

    #catch last line
    if current_line != '':
        ep_lines.append(current_line)
    
    return ep_lines

In [ ]:
tmp = process_marriedwc_episode(ep_raw)
print('Found {:d} lines'.format(len(tmp)))

In [ ]:
for ep in episode_links:
    print(ep)
    
    os.system('wget http://albundy.net/marriedaniac/ate/{:s}'.format(ep[0]))
    
    with open(ep[0].split('/')[1], mode='r', encoding='utf-8', errors='ignore') as file:
        ep_raw = file.readlines()
        
    ep_lines = process_marriedwc_episode(ep_raw)
    print('{:s}: Found {:d} lines and {:d} scenes'.format(ep[0],len(ep_lines),
                                                          len([l for l in ep_lines if l[:5]=='Scene'])))
    
    if len(ep_lines) > 150:
        with open('marriedwc_transcripts/refmt_{:s}.txt'.format(ep[1]), 'w') as f:
            for line in ep_lines:
                f.write(line+'\n')
        
        os.system('rm {:s}'.format(ep[0].split('/')[1]))

In [ ]:
remainders = glob.glob('[01]*html')
remainders = [r[:4] for r in remainders]
len(remainders)

In [ ]:
#Mop up the remainders - 
#  0218, 0305, 0607, 0608 have extra html so skip

for ep in [e for e in episode_links if e[1] in \
           remainders]:
    print(ep)
    
    with open(ep[0].split('/')[1], mode='r', encoding='utf-8', errors='ignore') as file:
        ep_raw = file.readlines()
        
    ep_lines = process_marriedwc_episode(ep_raw)
    print('{:s}: Found {:d} lines and {:d} scenes'.format(ep[0],len(ep_lines),
                                                          len([l for l in ep_lines if l[:5]=='Scene'])))
    
    if len(ep_lines) > 120:
        with open('marriedwc_transcripts/refmt_{:s}.txt'.format(ep[1]), 'w') as f:
            for line in ep_lines:
                f.write(line+'\n')
        
        os.system('rm {:s}'.format(ep[0].split('/')[1]))

## Scrubs

In [ ]:
import mechanize

In [ ]:
br = mechanize.Browser()
br.set_handle_robots(False)
br.set_handle_equiv(False)
br.addheaders = [('User-agent', 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1')]

In [ ]:
ep_list = br.open('https://scrubs.fandom.com/wiki/Category:Scrubs_Episodes').readlines()
ep_list = list(map(str, ep_list))

In [ ]:
def process_scrubs_episode(ep_raw):
    ep_raw = [l.replace('<b>','').replace('</b>','').replace('<p>','').replace('</p>','') for l in ep_raw]
    ep_raw = [l.replace('<br />','') for l in ep_raw]
    ep_raw = [l.strip() for l in ep_raw if (':' in l and 'href' not in l and '<' not in l and 'color-header' not in l) or \
              ('Act' in l and 'headline' in l) or \
              ('Scene' in l and 'headline' in l)]

    ep_raw = [l.replace("J.D.'s Narration","J.D.'s narration") for l in ep_raw]
    ep_raw = [l.replace("J.D.'s thoughts","J.D.'s narration") for l in ep_raw]
    ep_raw = [l.replace("J.D.'s Thoughts","J.D.'s narration") for l in ep_raw]
    ep_raw = [l.replace("J.D. Thoughts","J.D.'s narration") for l in ep_raw]
    ep_raw = [l.replace("JD","J.D.") for l in ep_raw]
    #remove speech intonations
    ep_raw = [re.sub('\[.*?\]', '', l) for l in ep_raw]
    ep_raw = [re.sub('\(.*?\)', '', l) for l in ep_raw]
    print(len(ep_raw))
    
    ep_proc = []
    scene_count = 0
    for l in ep_raw:
        if 'Preprocessor node count' in l:
            break

        if 'The following is a transcript' in l:
            continue
        if 'headline' in l and ('Act' in l or 'Scene' in l):
            scene_count += 1
            ep_proc.append('Scene %i' % scene_count)
        elif scene_count >= 1:
            ep_proc.append(l)

    return ep_proc

In [ ]:
season_counter, ep_counter = 0,0
for l in ep_list:
    if re.search('>Season \w*<', l) is not None and 'mw-headline' in l:
        season_counter += 1
        ep_counter = 0
    if 'href="https://vignette.wikia' in l and re.search('title="[\s\w]*"', l) is not None:
        ep_counter += 1
        ep_title = re.search('title="([\s\w]*)"', l).groups()[0]
        print(season_counter, ep_counter, ep_title)
        
        try:
            ep_text = br.open('https://scrubs.fandom.com/wiki/{:s}_transcript'.format(ep_title.replace(' ','_'))).readlines()
            ep_text = list(map(lambda l: str(l, 'utf-8'), ep_text))

            ep_lines = process_scrubs_episode(ep_text)
            #print(len(ep_lines))
            #print(pd.Series([l.split(':')[0] for l in ep_lines if ':' in l]).value_counts())

            if len(ep_lines) > 150:
                with open('scrubs_transcripts/refmt_{:02d}{:02d}.txt'.format(season_counter, ep_counter), 'w') as f:
                    for line in ep_lines:
                        f.write(line+'\n')
            else:
                print('MISSED ONE')
        except:
            print('No transcript found')
            
    #if season_counter > 1:
    #    break

Only get a few episodes in s6-9 so deleting these from scrubs_transcripts/.